# Barra CNE-6 Factor Calculation Function 

#### Size, Momentum, Volatility, Liquidity, Value 

In [1]:
import numpy as np
import pandas as pd
from datetime import *
from IPython.display import clear_output

#Size-----------------------------------------------------------------------------
def get_size_factors(dates, end_date, count, stocks, mv_df):
    """
    Calculates the size factors for given stock pool

    Parameters:
    - dates (DataFrame): Trading dates
    - end_date (str): End date
    - count (int): Number of trading dates considered
    - stocks (list): List of stock codes
    - mv_df (DataFrame): Circulating market cap for all stocks

    Return: A DataFrame containing date, code, LNCAP, and MIDCAP columns.
    
    """
    def MAD_winsorize(x, multiplier):
        """
        Winsorizes data using Median Absolute Deviation (MAD)

        """ 
        # Calculate the median and median absolute deviation
        x_M, x_MAD = np.nanmedian(x), np.nanmedian(np.abs(x - np.nanmedian(x)))
        # Find upper and lower bounds
        upper, lower = x_M + multiplier * x_MAD, x_M - multiplier * x_MAD 
        # Replace outliers with the bounds
        x[x > upper], x[x < lower] = upper, lower 
        return x
    
    def __reg(df):
        """
        Orthogonalize size factors using weighted regression

        """
        y = df['sub_MIDCAP'].values
        
        # Construct matrix with an intercept and independent variable
        X = np.c_[np.ones(len(y)), df['LNCAP'].values] 
        
        # Construct weight matrix using circulating market cap
        W = np.diag(np.sqrt(df['Circulating Market Cap']))
        
        # Find regression coefficient 
        beta = np.linalg.pinv(X.T @ W @ X) @ X.T @ W @ y 
        
        # Calculate residuals and apply winsorization           
        resi = MAD_winsorize(y - X@beta, multiplier=5)
        
        # Return the standardized residuals
        return pd.Series((resi - np.nanmean(resi)) / np.nanstd(resi), index=df['code'])
    
    # Determine the index of the  end_date 
    if pd.to_datetime(end_date) not in dates[0].values: 
        index_end = dates[0][dates[0] < pd.to_datetime(end_date)].idxmax()
    else:
        index_end =  dates[dates[0] == pd.to_datetime(end_date)].index.item()  
        
    # Determine start date
    start_date = dates[0].iloc[index_end - count] 
    
    #Filter data with given conditions
    datacur = mv_df[(start_date < mv_df['date']) & (mv_df['date'] < end_date) & mv_df['code'].isin(stocks)].dropna()
    
    # Scale the circulating market cap
    datacur['Circulating Market Cap'] /= 1e4  
    
    # Calculate factor LNCAP
    datacur['LNCAP'] = np.log(datacur['Circulating Market Cap'] + 1) 
    datacur['sub_MIDCAP'] = datacur['LNCAP']**3                
    
    # Group by date and apply the regression function
    MIDCAP = datacur.groupby('date').apply(__reg) 
    datacur = datacur.merge(MIDCAP.reset_index(name='MIDCAP'))
    print('Get size factors success!')
    return datacur[['date', 'code', 'LNCAP', 'MIDCAP']]


#Momentum-----------------------------------------------------------------------------
def get_momentum_factors(dates, end_date, count, stocks, cl,mv_df):
    """
    Calculate momentum factors for given stock data.

    Parameters:
    - dates: A DataFrame with trading dates.
    - end_date: The end date for the analysis.
    - count: The count used to determine the range of dates.
    - stocks: List of stock codes.
    - cl: DataFrame containing stock closing prices.
    - mv_df: DataFrame containing circulating market values for stocks.

    Returns:
    - factor: DataFrame with Short Term Reversal, Seasonality, Industry Momentum, Relative_strength, .
    """
    # Determine the index of the  end_date
    if pd.to_datetime(end_date) not in dates[0].values:
        index_end = dates[0][dates[0] < pd.to_datetime(end_date)].idxmax()
    else:
        index_end =  dates[dates[0] == pd.to_datetime(end_date)].index.item() 
        
    # Determine start date
    s = dates[0].iloc[index_end - count]
    
    # Calculate Short Term Reversal
    # Determine start date for calculation
    s1 = dates[0].iloc[index_end - count - 41]
    
    # Filter stock prices within the specified dates and stock pool
    price1 = cl
    price1 = price1[(s1 < price1['date']) & (price1['date'] < end_date)]
    price1 = price1[price1['code'].isin(stocks)]
    
    # Find return and change format
    price1['ret'] = price1['close'] / price1['pre_close'] - 1 
    ret = pd.pivot_table(price1, values='ret', index='date', columns='code')
    r_n = ret.rolling(21).mean().dropna(how='all')
    
    # Apply exponential weighting to returns
    W = get_exponent_weight(window=21, half_life=5)
    STREV = []
    for i in range(len(r_n)-20):
        # Calculate log returns
        tmp = np.log(1 + r_n.iloc[i:i+21, :].copy())
        # Calculate weighted sum of log returns
        tmp = pd.Series(np.sum(W.reshape(-1, 1)*tmp.values, axis=0), name=tmp.index[-1], index=tmp.columns)
        STREV.append(tmp)
    STREV = pd.concat(STREV, axis=1).T
    STREV.index.name = 'date'
    STREV = pd.melt(STREV.reset_index(), id_vars='date', value_name='Short_Term_reversal').dropna()

    # Calculate Seasonality Factor
    trade_date = dates[(s < dates[0]) & (dates[0] < end_date)]
    season = {}
    for td in trade_date[0]:
        r_y = []
        for i in range(1, 4):
            # Find the closest date that is greater than the calculated s_
            s_ = td - pd.Timedelta(days=365*i)
            # Calculate e_ based on s_
            if s_ not in dates[0].values:
                closest_date_index = dates[0][dates[0] > pd.to_datetime(s_)].idxmin()
                s_ = dates[0].iloc[closest_date_index]  
                e_ = dates[0].iloc[closest_date_index + 21]
            else:
                s_p = dates[dates[0] == s_].index
                e_ = dates[0].iloc[s_p + 21].values[0]
            
            p_ = cl[['date','code','close']]
            # Filter stock prices within the specified date range
            p_ = p_[(s_ < p_['date']) & (p_['date'] < e_)]
            p_ = p_[p_['code'].isin(stocks)]
            # Create a pivot table of stock prices and calculate returns
            p_ = pd.pivot_table(p_, index='date', columns='code', values='close').ffill()
            r_y.append(p_.iloc[-1, :] / p_.iloc[0, :] - 1)
        season[pd.to_datetime(td)] = pd.concat(r_y, axis=1).mean(axis=1)
        
    season = pd.DataFrame(season).T
    season.index.name = 'date'
    season = pd.melt(season.reset_index(), id_vars='date', value_name='Seasonality')
    factor = pd.merge(STREV, season)
    
    # Calculate Industry Momentum
    # find start date for calculation
    s2 = dates[0].iloc[index_end - count - 126]
    
    # filter prices within specified dates and stock pools
    price2 = cl
    price2 = price2[(s2 < price2['date']) & (price2['date'] < end_date)]
    price2 = price2[price2['code'].isin(stocks)]
    
    # find return
    price2['ret'] = price2['close'] / price2['pre_close'] - 1 
    ret = pd.pivot_table(price2, index='date', columns='code', values='ret')
    
    # Apply exponential weighting to returns
    W = get_exponent_weight(window=126, half_life=21)
    
    RS = {}
    for i in range(len(ret)-125):
        # Extract a 126-day window of returns for analysis
        tmp = ret.iloc[i:i+126, :].copy()
        # Filter out stocks with more than 10% missing data and fill NaN values with 0
        tmp = tmp.loc[:, tmp.isnull().sum(axis=0) / 252 <= 0.1].fillna(0.)
        # Calculate log returns
        tmp = np.log(1 + tmp)
        # Calculate RS values
        RS[tmp.index[-1]] = pd.Series(np.sum(W.reshape(-1, 1)*tmp.values, axis=0), index=tmp.columns)
    RS = pd.DataFrame(RS).T
    RS.index.name = 'date'
    RS = pd.melt(RS.reset_index(), id_vars='date', value_name='RS').dropna().reset_index(drop=True)
    
    #filter circulating market cap within specified dates and stock pools
    val = mv_df
    val = val[(s < val['date']) & (val['date'] < end_date)]
    val = val[val['code'].isin(stocks)]
    
    RS = pd.merge(RS, val)
    # Extract the month and year from the 'date' column and store it as 'mon'
    RS['mon'] = RS['date'].apply(lambda x: x.strftime("%Y-%m-01"))
    
    # Extract the month and year from the 'date' column and store it as 'mon'
    RS['c'] = np.sqrt(RS['Circulating Market Cap'])

    def __industry_RS(x):
        ind_RS = x.groupby('industry').apply(
            lambda y: y['RS'].dot(y['c']) / y['c'].sum()
        )
        # Group data by industry and calculate industry-specific RS
        ind_RS.name = 'ind_RS'
        ind_RS = ind_RS.reset_index()
        x = pd.merge(x, ind_RS)
        # Calculate the Industry Momentum
        x['Industry_Momentum'] = x['ind_RS'] - x['RS']
        return x[['code', 'Industry_Momentum']].set_index('code')
    
    INDMOM = []
    
    # filter industry data
    ind = industry_long[industry_long['date'] == pd.to_datetime('2022-12-30')]
    ind = ind[['code','industry']]
    # Group the RS DataFrame by month ('mon')
    for m, tmp_RS in RS.groupby('mon'):
        tmp_RS = pd.merge(tmp_RS, ind)
        print(tmp_RS)
        # Calculate Industry Momentum
        INDMOM.append(tmp_RS.groupby('date').apply(__industry_RS).reset_index())
    INDMOM = pd.concat(INDMOM).reset_index(drop=True)
#     print(INDMOM)
    factor = factor.merge(INDMOM)

    # Calculate Relative Strength
    s3 = dates[0].iloc[index_end - count - 262]
    W = get_exponent_weight(window=252, half_life=126)
    
    # filter price and find return 
    price3 = cl_long
    price3 = price3[(s3 < price3['date']) & (price3['date'] < end_date)]
    price3 = price3[price3['code'].isin(stocks)]
    price3['ret'] = np.log(price3['close']) - np.log(price3['pre_close'])
    ret = pd.pivot_table(price3, index='date', columns='code', values='ret')

    Relative_strength = {}
    for i in range(len(ret) - 251):
        # Extract a 252-day window of returns for analysis
        tmp = ret.iloc[i:i+252, :]
        # Filter out stocks with more than 10% missing data
        tmp = tmp.loc[:, tmp.isnull().sum(axis=0) / 252 <= 0.1].fillna(0.)
        # Calculate the weighted sum of returns for the selected window
        np.sum(W.reshape(-1, 1)*tmp.values, axis=0)
        # Calculate Relative Strength
        Relative_strength[tmp.index[-1]] = pd.Series(np.sum(W.reshape(-1, 1)*tmp.values, axis=0), index=tmp.columns)
    Relative_strength = pd.DataFrame(Relative_strength).T
    Relative_strength.index.name = 'date'
    Relative_strength = Relative_strength.rolling(11).mean().dropna(how='all')
    Relative_strength = pd.melt(Relative_strength.reset_index(), id_vars='date', value_name='Relative_strength').dropna().reset_index(drop=True)
    #print(Relative_strength)
    factor = factor.merge(Relative_strength)
    clear_output()
    print ('Get size factors success!')
    return factor

#Volatility-----------------------------------------------------------------------------
def get_volatility_factors(dates, end_date, count, window, half_life ,stocks, cl_long,dqmv_long, wind_price):
    """
    Calculate volatility factors for given stock data.

    Parameters:
    - dates: A DataFrame with trading dates.
    - end_date: The end date for the analysis.
    - count: The count used to determine the range of dates.
    - window: The size of the rolling window
    - half_life: The half-life for exponential weighting
    - stocks: List of stock codes.
    - cl: DataFrame containing stock closing prices.
    - dqmv_df: DataFrame containing circulating market values for stocks.

    Returns:
    - factor: DataFrame with Beta, Hist sigma, Daily std, Cumulative range
    """
    
    # find index of end date
    if pd.to_datetime(end_date) not in dates[0].values:
        index_end = dates[0][dates[0] < pd.to_datetime(end_date)].idxmax()
    else:
        index_end =  dates[dates[0] == pd.to_datetime(end_date)].index.item() 
    
    # find start date
    s = dates[0].iloc[index_end - count]
    start_date = dates[0].iloc[index_end - count - window]
    
    # filter stock prices
    price = cl_long
    price = price[(start_date < price['date']) & (price['date'] < end_date)]
    price = price[price['code'].isin(stocks)]
    
    # filter Wind All China Index prices 
    wind = wind_price[(start_date < wind_price['date']) & (wind_price['date'] < end_date)].copy()
    wind.loc[:, 'code'] = '881001'
    price = pd.concat([price, wind]).reset_index(drop=True)
    price = price.drop('Unnamed: 0', axis=1)
    
    # find return
    price['ret'] = price['close'] / price['pre_close'] - 1
    ret = pd.pivot_table(price, values='ret', index='date', columns='code')
    
    # Calculate Lambda (decay factor) and initialize weights 'W' for exponential weighting
    L, Lambda = 0.5**(1/half_life), 0.5**(1/half_life)
    W = []
    for i in range(window):
        W.append(Lambda)
        Lambda *= L
    W = W[::-1]
    
    # Calculate Lambda (decay factor) and initialize weights 'W' for exponential weighting
    beta, hist_sigma = [], []
    for i in range(len(ret)-window+1):
        tmp = ret.iloc[i:i+window, :].copy()
        W_full = np.diag(W)
        Y_full = tmp.dropna(axis=1).drop(columns='881001')
        idx_full, Y_full = Y_full.columns, Y_full.values
        X_full = np.c_[np.ones((window, 1)), tmp.loc[:, '881001'].values]
        
        # Calculate BETA and hist_sigma for the full dataset
        beta_full = np.linalg.pinv(X_full.T@W_full@X_full)@X_full.T@W_full@Y_full
        hist_sigma_full = pd.Series(np.std(Y_full - X_full@beta_full, axis=0), index=idx_full, name=tmp.index[-1])
        beta_full = pd.Series(beta_full[1], index=idx_full, name=tmp.index[-1])
        
        beta_lack, hist_sigma_lack = {}, {}
        for c in set(tmp.columns) - set(idx_full) - set('881001'):
            tmp_ = tmp.loc[:, [c, '881001']].copy()
            tmp_.loc[:, 'W'] = W
            tmp_ = tmp_.dropna()
            W_lack = np.diag(tmp_['W'])
            if len(tmp_) < half_life:
                continue
            X_lack = np.c_[np.ones(len(tmp_)), tmp_['881001'].values]
            Y_lack = tmp_[c].values
            
            # Calculate BETA and hist_sigma for stocks with incomplete data
            beta_tmp = np.linalg.pinv(X_lack.T@W_lack@X_lack)@X_lack.T@W_lack@Y_lack
            hist_sigma_lack[c] = np.std(Y_lack - X_lack@beta_tmp)
            beta_lack[c] = beta_tmp[1]
        beta_lack = pd.Series(beta_lack, name=tmp.index[-1])
        hist_sigma_lack = pd.Series(hist_sigma_lack, name=tmp.index[-1])
        beta.append(pd.concat([beta_full, beta_lack]).sort_index())
        hist_sigma.append(pd.concat([hist_sigma_full, hist_sigma_lack]).sort_index())
    beta = pd.concat(beta, axis=1).T
    beta = pd.melt(beta.reset_index(), id_vars='index').dropna()
    beta.columns = ['date', 'code', 'BETA']
    hist_sigma = pd.concat(hist_sigma, axis=1).T
    hist_sigma = pd.melt(hist_sigma.reset_index(), id_vars='index').dropna()
    hist_sigma.columns = ['date', 'code', 'Hist_sigma']
    factor = pd.merge(beta, hist_sigma)
    
    # Calculate the Exponentially Weighted Moving Average (EWMA) of daily standard deviation (Daily_std)
    # Initialize the standard deviation and variance
    init_std = ret.std(axis=0)
    L = 0.5**(1/42)
    init_var = ret.var(axis=0)
    tmp = init_var.copy()
    daily_std = {}
    
    # calculate daily_std
    for t, k in ret.iterrows():
        tmp = tmp*L + k**2*(1-L)
        daily_std[t] = np.sqrt(tmp)
        tmp = tmp.fillna(init_var)
    daily_std = pd.DataFrame(daily_std).T
    daily_std.index.name = 'date'
    daily_std = daily_std.loc[s:end_date, :]
    daily_std = pd.melt(daily_std.reset_index(), id_vars='date', value_name='Daily_std').dropna()

    factor = factor.merge(daily_std)

    close = pd.pivot_table(price, values='close', index='date', columns='code').fillna(method='ffill', limit=10)

    pre_close = pd.pivot_table(price, values='pre_close', index='date', columns='code').fillna(method='ffill', limit=10)
    idx = close.index
    CMRA = {}
    
    # Calculate the Cumulative Moving Range Average (CMRA)
    for i in range(len(close)-window+1):
        close_ = close.iloc[i:i+window, :]
        pre_close_ = pre_close.iloc[i, :]
        pre_close_.name = pre_close_.name - pd.Timedelta(days=1)
        pre_close_1 = pd.DataFrame([pre_close_.values], columns=pre_close_.index, index=[pre_close_.name])
        close_ = pd.concat([close_, pre_close_1], axis=1)
        close_ = close_.sort_index().iloc[list(range(0, 253, 21)), :]
        r_tau = close_.pct_change().dropna(how='all')
        Z_T = np.log(r_tau+1).cumsum(axis=0)
        CMRA[idx[i+window-1]] = Z_T.max(axis=0) - Z_T.min(axis=0)
    CMRA = pd.DataFrame(CMRA).T
    CMRA.index.name = 'date'
    CMRA = pd.melt(CMRA.reset_index(), id_vars='date', value_name='Cumulative_range').dropna()
    factor = factor.merge(CMRA)
    # Remove rows where 'Cumulative_range' is equal to 0
    factor = factor[factor['Cumulative_range'] != 0] 
    
    # Remove rows of Wind All China Index
    factor = factor[factor['code'] != '881001']

    clear_output()
    print ('Get size factors success!')
    return factor.sort_values(by=['date', 'code'])

#Liquidity-----------------------------------------------------------------------------
def get_liquidity_factors(end_date, count, stocks,turn):
    """
    Calculate liquidity factors for given stock data.

    Parameters:
    - dates: A DataFrame with trading dates.
    - end_date: The end date for the analysis.
    - count: The count used to determine the range of dates.
    - turn: A dataframe with turnover-ratio 

    Returns:
    - factor: DataFrame with Monthly share turnover, Quarterly share turnover, Annual share turnover
    """
    
    # find the index of end_date
    if pd.to_datetime(end_date) not in dates[0].values:
        index_end = dates[0][dates[0] < pd.to_datetime(end_date)].idxmax()
    else:
        index_end =  dates[dates[0] == pd.to_datetime(end_date)].index.item() 
    
    # find start date
    s = dates[0].iloc[index_end - count]
    
    # find start date of data for calculation
    start_date = dates[0].iloc[index_end - count - 252]
    
    # filter turnover ratio data
    tmp = turn_long
    tmp = tmp[(start_date < tmp['date']) & (tmp['date'] < end_date)]
    tmp = tmp[tmp['code'].isin(stocks)]
    tmp['turnover_ratio'] /= 100.
    tmp = pd.pivot_table(tmp, index='date', columns='code', values='turnover_ratio')

    # Calculate monthly, quarterly, and annual share turnover factor
    monthly_share_turnover = np.log(tmp.rolling(21).sum()+1e-10)
    idx = list(range(20, 252, 21))
    quarterly_share_turnover, annual_share_turnover = {}, {}

    for i in range(len(tmp) - 251):
        # get t for calculation
        t = tmp.index[i+251]
        # taking the exponential of the monthly share turnover values for 252 days
        mst = np.exp(monthly_share_turnover.iloc[i:i+252, :].iloc[idx, :])
        # taking the logarithm of the mean of the last 3 monthly values
        quarterly_share_turnover[t] = np.log(mst.iloc[-3:, :].mean(axis=0))
        # taking the logarithm of the mean of a year
        annual_share_turnover[t] = np.log(mst.mean(axis=0))
    quarterly_share_turnover = pd.DataFrame(quarterly_share_turnover).T
    annual_share_turnover = pd.DataFrame(annual_share_turnover).T
    quarterly_share_turnover.index.name = 'date'
    annual_share_turnover.index.name = 'date'
    monthly_share_turnover = monthly_share_turnover.loc[s:end_date, :]
    monthly_share_turnover = pd.melt(monthly_share_turnover.reset_index(), id_vars='date', value_name='Monthly_share_turnover').dropna()
    quarterly_share_turnover = pd.melt(quarterly_share_turnover.reset_index(), id_vars='date', value_name='Quarterly_share_turnover').dropna()
    annual_share_turnover = pd.melt(annual_share_turnover.reset_index(), id_vars='date', value_name='Annual_share_turnover').dropna()
    
    factor = monthly_share_turnover.merge(quarterly_share_turnover).merge(annual_share_turnover)

    # Calculate annualized traded value ratio
    window, half_life = 252, 63
    L, Lambda = 0.5**(1/half_life), 0.5**(1/half_life)
    W = []
    for i in range(window):
        W.append(Lambda)
        Lambda *= L
    W = np.array(W[::-1])/np.mean(W)
    
    annualized_traded_value_ratio = []

    for i in range(len(tmp) - 251):
        tmp_ = tmp.iloc[i:i + 252, :].copy()
        values = tmp_.values * W.reshape(-1, 1)

        # Replace NaN values with zeros
        values = np.nan_to_num(values)

        mean_values = np.nanmean(values, axis=0)
        annualized_traded_value_ratio.append(
            pd.Series(mean_values, index=tmp.columns, name=tmp_.index[-1])
        )
    annualized_traded_value_ratio = pd.concat(annualized_traded_value_ratio, axis=1).T * window
    annualized_traded_value_ratio.index.name = 'date'
    annualized_traded_value_ratio = pd.melt(annualized_traded_value_ratio.reset_index(), id_vars='date', value_name='Annualized_traded_value_ratio').dropna()
    factor = factor.merge(annualized_traded_value_ratio)
    return factor


#Value-----------------------------------------------------------------------------
def get_value_factors(end_date, count,stocks, ratio, cl,wind):
    """
    Calculate value factors for given stock data.

    Parameters:
    - end_date: The end date for the analysis.
    - count: The count used to determine the range of dates.
    - stocks: The list of stock codes
    - ratio: Dataframe of pb, pe and pcf ratio
    - cl - Dataframe of close price
    - wind - Dataframe of Wind All China Index prices

    Returns:
    - factor: DataFrame with Book to price, EP ratio, Cash earning to price, Long term relative strength, Long term historical alpha
    """
    # find end date index
    if pd.to_datetime(end_date) not in dates[0].values:
        index_end = dates[0][dates[0] < pd.to_datetime(end_date)].idxmax()
    else:
        index_end =  dates[dates[0] == pd.to_datetime(end_date)].index.item() 
    
    # find start date
    s = dates[0].iloc[index_end - count]
    
    # filter all ratio values
    val = ratio
    val = val[val['code'].isin(stocks)]
    val = val[(s < val['date']) & (val['date'] < pd.to_datetime(end_date))]
    
    # Calculate Book to Price, Earnings to Price, and Cash Earnings to Price
    val['Book_to_price'] = 1 / val['pb_ratio']
    val['Earning_to_price'] = 1 / val['pe_ratio']
    val['Cash_earning_to_price'] = 1 / val['pcf_ratio']
    factor = val[['code', 'date', 'Book_to_price', 'Earning_to_price', 'Cash_earning_to_price']]
    
    # Define a function to calculate cumulative mean
    def __cummean(x):
        f_ = []
        def __sub_cummean(y, f_):
            f_ += y['np'].values.tolist()
            if len(f_)<5:
                return np.nan
            return np.nanmean(f_)
        np_std = x.groupby('time').apply(lambda z: __sub_cummean(z, f_))
        np_std.name = 'np_mean'
        return np_std.dropna()

    window = 750
    half_life = 260
    
    # Calculate Long-term Relative Strength
    s3 = dates[0].iloc[index_end - count - 750 - 12]
    W = get_exponent_weight(window=window, half_life=260)
    price = cl
    price = price[(s3 < price['date']) & (price['date'] < end_date)]
    price = price[price['code'].isin(stocks)]
    price['ret'] = np.log(price['close']) - np.log(price['pre_close'])
    ret = pd.pivot_table(price, index='date', columns='code', values='ret')

    Relative_strength = {}
    for i in range(len(ret) - window-1):
        tmp = ret.iloc[i:i+window, :]
        tmp = tmp.loc[:, tmp.isnull().sum(axis=0) / window <= 0.1].fillna(0.)
        np.sum(W.reshape(-1, 1)*tmp.values, axis=0)
        Relative_strength[tmp.index[-1]] = pd.Series(np.sum(W.reshape(-1, 1)*tmp.values, axis=0), index=tmp.columns)
    Relative_strength = pd.DataFrame(Relative_strength).T
    Relative_strength.index.name = 'date'
    Relative_strength = Relative_strength.rolling(11).mean().dropna(how='all').mul(-1)
    Relative_strength = pd.melt(Relative_strength.reset_index(), id_vars='date', value_name='Longterm_Relative_strength')\
        .dropna().reset_index(drop=True)

    # Calculate Long-term Alpha (similar to volatility beta cauculation)
    price = cl
    price = price[(s3 < price['date']) & (price['date'] < end_date)]
    price = price[price['code'].isin(stocks)]
    wind = wind_price[(s3 < wind_price['date']) & (wind_price['date'] < end_date)].copy()
    wind.loc[:, 'code'] = '881001'
    price = pd.concat([price, wind]).reset_index(drop=True)
    price = price.drop('Unnamed: 0', axis=1)
    price['ret'] = price['close'] / price['pre_close'] - 1
    ret = pd.pivot_table(price, values='ret', index='date', columns='code')
    W = get_exponent_weight(window=window, half_life=half_life)
    
    alpha = []
    for i in range(len(ret)-window+1):
        tmp = ret.iloc[i:i+window, :].copy()#
        W_full = np.diag(W)#
        Y_full = tmp.dropna(axis=1).drop(columns='881001')#

        idx_full, Y_full = Y_full.columns, Y_full.values#
        X_full = np.c_[np.ones((window, 1)), tmp.loc[:, '881001'].values]#

        alpha_full = np.linalg.pinv(X_full.T@W_full@X_full)@X_full.T@W_full@Y_full
        alpha_full = pd.Series(alpha_full[1], index=idx_full, name=tmp.index[-1])

        alpha_lack = {}
        for c in set(tmp.columns) - set(idx_full) - set('881001'):
            tmp_ = tmp.loc[:, [c, '881001']].copy()
            tmp_.loc[:, 'W'] = W
            tmp_ = tmp_.dropna()
            W_lack = np.diag(tmp_['W'])
            if len(tmp_) < half_life:
                continue
            X_lack = np.c_[np.ones(len(tmp_)), tmp_['881001'].values]
            Y_lack = tmp_[c].values
            alpha_tmp = np.linalg.pinv(X_lack.T@W_lack@X_lack)@X_lack.T@W_lack@Y_lack
            alpha_lack[c] = alpha_tmp[1]
        alpha_lack = pd.Series(alpha_lack, name=tmp.index[-1])
        alpha.append(pd.concat([alpha_full, alpha_lack]).sort_index())
        
    alpha = pd.concat(alpha, axis=1).T
    alpha = pd.melt(alpha.reset_index(), id_vars='index').dropna()
    alpha.columns = ['date', 'code', 'alpha']
    alpha = alpha[alpha['code'] != '881001']
    factor = pd.merge(factor,Relative_strength, on = ['code','date'], how = 'outer' )
    factor = pd.merge(factor, alpha, how = 'outer')
    factor = factor[factor['code'] != '881001'].reset_index()
    return factor

def get_exponent_weight(window, half_life, is_standardize=True):
    # Calculate the decay factors L and Lambda.
    L, Lambda = 0.5**(1/half_life), 0.5**(1/half_life)
    W = []
    # Loop through the specified window size to calculate exponential weights
    for i in range(window):
        W.append(Lambda)                                                         
        Lambda *= L # Update Lambda's value
    W = np.array(W[::-1])
    if is_standardize:
        W /= np.sum(W) # Normalize the weights
    return W